In [ ]:
# Enable hot-reloading so if you edit src/train.py, it updates here immediately
%load_ext autoreload
%autoreload 2



GPU Available: True


Initializing mBART on cuda...


Input Shape: torch.Size([128])
Labels Shape: torch.Size([128])


/home/ubuntu/miniconda3/envs/deeplearning-gpu/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


/tmp/ipykernel_12923/1370919502.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedSeq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedSeq2SeqTrainer(


/home/ubuntu/miniconda3/envs/deeplearning-gpu/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/ubuntu/miniconda3/envs/deeplearning-gpu/lib/python3.10/site-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
1500,0.370700,0.353512
3000,0.349500,0.339190
4500,0.340300,0.328251
6000,0.257600,0.328729
7500,0.262400,0.327250
9000,0.269000,0.322399
10500,0.260200,0.322384
12000,0.215400,0.331449
13500,0.212500,0.330756
15000,0.206200,0.330291


/home/ubuntu/miniconda3/envs/deeplearning-gpu/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/home/ubuntu/miniconda3/envs/deeplearning-gpu/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/ubuntu/miniconda3/envs/deeplearning-gpu/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is d

Model saved to models/mbart_neutralizer_en_v1
Output: the regime failed to act.


Loading fine-tuned model for final verification...
Initializing mBART on cuda...


The tokenizer you are loading from 'models/mbart_neutralizer_en_v1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.



==================== PHASE 1 COMPLETE: ENGLISH BASELINE ====================

Original: The radical regime failed to act on the crisis.
Neutral:  the iranian government failed to act on the crisis.
--------------------------------------------------
Original: The controversial politician foolishly denied the allegations.
Neutral:  the controversial politician denied the allegations.
--------------------------------------------------
Original: He exposed the senator's corruption.
Neutral:  he accused the senator's corruption.
--------------------------------------------------

If the 'Neutral' outputs removed the biased words (radical, foolishly, exposed)
while keeping the facts, Phase 1 is SUCCESSFUL.


In [ ]:
from src.final_models import PivotBaseline, ZeroShotNeutralizer, SyntheticNeutralizer
import pandas as pd

# 1. Load Data
test_df = pd.read_csv("data/processed/test_chinese_gold.csv")
inputs = test_df["Chinese_Biased"].tolist()

# 2. Initialize Models
# (This might take VRAM, so maybe run one at a time if you crash)
models = {
    "Baseline (Pivot)": PivotBaseline(),
    "Model 2 (ZeroShot)": ZeroShotNeutralizer(),
    "Model 3 (Synthetic)": SyntheticNeutralizer(),
}

# 3. Run Experiments
results = {}
for name, model in models.items():
    print(f"Running {name}...")
    # Batch processing 50 items is fast enough to do in a loop
    results[name] = [model.debias(text) for text in inputs]

# 4. Save Results for Evaluation
results_df = pd.DataFrame(results)
results_df["input"] = inputs
results_df["gold"] = test_df["Chinese_Neutral_Gold"]
results_df.to_csv("results/final_predictions.csv", index=False)

print("Inference Complete! Ready for evaluate.py")